In [ ]:
Los indices que se van a calcular son los siguientes: 
{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {adi} {aci} {te} {sme} {ndsi} {r} {me} {mba} {bnt} {md} {fm} {we}    


In [2]:
#Feature extraction of acoustics index

from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display
from time import time #importamos la función time para capturar tiempos

import sklearn
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')
import soundfile as sf
from Indices import *


In [3]:
ruta_archivo="/home/david/Documentos/Datos Selecccionados para analisis de variables agosto 29transformcion/Alta/Cluster1/13654-302143__0__20170123_001000.wav"


In [48]:
'''
La distribucion de carpetas funciona asi: 
1)dentro de una carpeta alta,baja,media se encuentran varios tipos de subcarpetas llamadas clusters. 
Lo importante es que este por lo menos dos carpetas por encima, el archivo datos.csv va a quedar guardado 
en la carpeta de arriba. 
'''
%cd /home/david/Documentos/Datos Selecccionados para analisis de variables agosto 29transformcion/Media
import os
os.getcwd()
genres=os.listdir()
genres

/home/david/Documentos/Datos Selecccionados para analisis de variables agosto 29transformcion/Media


['Cluster3', 'Cluster4']

In [49]:
titulos = ["ADI", "ACItf", "BI", "TE", "ESM", "NDSI", "P", "M", "MID" "BNT", "MD",
                       "FM", "SF", "RMS"]
header = 'filename chroma_stft  spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'

header = header.split()

header += titulos
header += ['label']
##Variables para indices
tipo_ventana = "hann"
sobreposicion = 0
tamano_ventana = 512
nfft= 512
bio_band = (2000, 8000)
tech_band = (200, 1500)

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for g in genres:
    for filename in os.listdir(f'{g}'):
        clear_output()
        print(g)
        print(filename)
        songname = f'{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)

        print(spec_bw)
        x, Fs = sf.read(songname)
        audio = x
        nmin = len(audio) // (60 * Fs)

        ## indices acusticos
        f, t, s = signal.spectrogram(audio, Fs, window=tipo_ventana, nperseg=nmin * tamano_ventana,
                                 mode="magnitude", \
                                 noverlap=sobreposicion, nfft=nmin * nfft)
        nmin = len(audio) // (60 * Fs)


        adi=ADI(s, 10000, 1000, -50)
        print(adi)
        aci=ACItf(audio, Fs, 5, s)
       # beta=beta(s, f, bio_band)//nmin
        te=temporal_entropy(audio, Fs)
        sme=spectral_maxima_entropy(s, f, 482, 8820)
        ndsi=NDSI(s, f, bio_band, tech_band)
        r=rho(s, f, bio_band, tech_band)
        me=median_envelope(audio, Fs, 16)
        #number_of_peaks(s, f, 10 * nmin)
        mba=mid_band_activity(s, f, 450, 3500)
        #np.mean(background_noise_freq(s))
        bnt=background_noise_time(wav2SPL(audio, -11, 9, 0.707), 5)
        md=musicality_degree(audio, Fs, tamano_ventana, nfft, tipo_ventana, sobreposicion)
        fm=frequency_modulation(s)
        we=wiener_entropy(audio, tamano_ventana, nfft, tipo_ventana, sobreposicion)
        #rms=rms(audio)

        
        
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {adi} {aci} {te} {sme} {ndsi} {r} {me} {mba} {bnt} {md} {fm} {we}'    
       # to_append += {adi} {aci} {beta}{te}{sme}{ndsi}{r}{me}{mba}{bnt}{md}{fm}{we}{rms}
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

Cluster4
ESPAN5067__0__20160225_072000.wav
[[2944.02345349 2852.51192336 2729.50839933 ... 2868.3391811
  2773.72848065 2786.77539131]]
0.9621405371155062


In [52]:
data = pd.read_csv('/home/david/Documentos/Datos Selecccionados para analisis de variables agosto 29transformcion/datosabm.csv')
data.head()


,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,ADI,ACItf,BI,TE,ESM,NDSI,P,M,MIDBNT,MD,FM,SF,cluster,label
0,13654-SM5071__0__20170202_034000.wav,0.690695,1657.156469,2543.075990,5489.708471,0.000000,1.695777,45163.038106,0.999886,0.966630,0.921355,24.430695,789.255191,0.021858,-100.35151,-8.764474,40.122620,0.104645,Cluster7,Alta
1,13654-SM5071__0__20170214_125000.wav,0.903377,1637.015007,3035.014862,5725.157047,0.000000,0.486196,44696.404254,0.999970,0.921059,0.349031,2.072345,780.001271,0.127244,-100.35151,-9.474842,40.178186,0.375743,Cluster7,Alta
2,13654-SM5071__0__20170124_155000.wav,0.887091,2111.695309,3435.153118,7233.906258,0.000003,0.603109,44632.051399,0.999923,0.885995,-0.005463,0.989133,781.472125,0.062451,-100.35151,-10.954779,40.193273,0.172189,Cluster7,Alta
3,13654-SM5071__0__20170216_062000.wav,0.904979,891.760079,2202.454371,1938.763277,0.000000,1.468066,44912.396001,0.999987,0.985800,0.454960,2.669455,788.103558,0.127244,-100.35151,-9.099079,40.065809,0.609970,Cluster7,Alta
4,13654-SM5071__0__20170216_161000.wav,0.905819,707.012431,1926.155526,949.019261,0.000000,0.470851,44720.803509,0.999981,0.958402,-0.443680,0.385349,778.089187,0.123341,-100.35151,-8.511960,40.137089,0.334119,Cluster7,Alta
